In [19]:
import pandas as pd

# First dataset

In [20]:
epidemiology_df = pd.read_excel('situacion-epidemiologica-coronavirus-en-castilla-y-leon.xlsx').drop(['codigo_ine', 'Posicion'], axis=1).sort_values(by='fecha').reset_index(drop=True)

Problems found with the first dataset ('situacion-epidemiologica-coronavirus-en-castilla-y-leon'):
- Individual problems: 
    + A. Poor assigned names to the different variables that leads to confussion (and no descripcions in the source) -> SOL: Analysis of the data to determine its true meaning
    + B. For some variables the daily and accumulated values are registered, but not for others -> SOL: Preprocessing of the data to generate both values for each variable
- Problems for the merging with the other dataset:
    + C. Different values in the province variable (this dataset has the 'Total_CyL' value missing) -> SOL: Preprocessing of the data to generate the 'Total_CyL' values that summarize the information from the whole autonomous community.

### Solving problem A
##### Assign proper names to the variables

In [21]:
# Change the names and translate
epidemiology_df.columns = ['date', 'province', 'total_positives', 'new_positives', 'total_discharges', 'total_deceases']

### Solving problem B
##### Obtain the daily values of each variable from the accumulated values

In [22]:
# Create the new variables for daily data
epidemiology_df['new_discharges'] = 0
epidemiology_df['new_deceases']   = 0

# Obtain the daily data by substracting the accumulated values from consecutive days
for current_province in epidemiology_df['province'].unique():
    
    rows_current_province = epidemiology_df['province'] == current_province
    data_current_province = epidemiology_df[rows_current_province]
    
    epidemiology_df.loc[rows_current_province,'new_discharges'] = data_current_province['total_discharges'].diff()
    epidemiology_df.loc[rows_current_province,'new_deceases']   = data_current_province['total_deceases'].diff()

# Fix the values for the first day (it has no previous day to substract in the previous step)
epidemiology_df.loc[0:9,'new_discharges'] = epidemiology_df[0:9]['total_discharges']
epidemiology_df.loc[0:9,'new_deceases']   = epidemiology_df[0:9]['total_deceases']

### Solving problem C
##### Generate the 'TotalCyL' value for the 'province' variable by adding the daily information of each of the provinces

In [23]:
# Group the data by date and add the data of all the provinces
epidemiology_CyL_df = epidemiology_df.groupby(by='date').sum()

# Fix the index to keep the date and add the 'province' value
epidemiology_CyL_df = epidemiology_CyL_df.reset_index()
epidemiology_CyL_df['province'] = 'TotalCyL'

# Concatenate the new rows to the previous dataset
epidemiology_df = pd.concat([epidemiology_df, epidemiology_CyL_df]).sort_values(by='date').reset_index(drop=True)

### Final state of the dataset

In [34]:
epidemiology_df

,date,province,total_positives,new_positives,total_discharges,total_deceases,new_discharges,new_deceases
0,2020-03-13,Burgos,90.0,29.0,0.0,1.0,0.0,1.0
1,2020-03-13,Valladolid,11.0,7.0,0.0,0.0,0.0,0.0
2,2020-03-13,Palencia,1.0,1.0,0.0,0.0,0.0,0.0
3,2020-03-13,León,32.0,22.0,0.0,0.0,0.0,0.0
4,2020-03-13,Ávila,3.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
7655,2022-04-19,Ávila,43704.0,87.0,2460.0,469.0,3.0,0.0
7656,2022-04-19,Valladolid,166979.0,213.0,10009.0,1510.0,9.0,1.0
7657,2022-04-19,León,119312.0,328.0,7261.0,1481.0,15.0,2.0
7658,2022-04-19,Soria,28118.0,73.0,1665.0,330.0,2.0,0.0


# Second dataset

In [25]:
vacunation_df = pd.read_excel('personas-vacunadas-covid.xlsx').sort_values(by='fecha').reset_index(drop=True)

Problems found with the first dataset ('personas-vacunadas-covid'):
- Individual problems: 
    + A. Some variables have been registered since the creation of the dataset and others have been included later, derivating in confusing names and numerous missing values -> SOL: Analysis of the data to determine its true meaning and the cause of the missings
    + B. For some variables the absolute and percentage values are registered, but not for others -> SOL: Preprocessing of the data to generate both values for each variable
- Problems for the merging with the other dataset:
    + C. Different values in the province variable (this dataset has the 'Total_CyL' value ) -> SOL: Solved in the previous dataset

### Solving problem A
##### 1. Assign proper names to the variables

In [26]:
# Change the names and translate
vacunation_df.columns = ['date', 'province', 'new_doses_administered', 'new_persons_two_doses', 'new_persons_one_dose', 'percentage_persons_one_dose', 'percentage_persons_two_doses',  'new_persons_three_doses', 'percentage_persons_three_doses']

##### 2. Determine the meaning of the missings and set values if needed

In this dataset we have 2 different cases:
- 'Percentage_persons_one_dose'and 'percentage_persons_two_doses' were not measured since the begining but the data can be obtained from the original variables.
- 'New_persons_three_doses' and 'percentage_persons_three_doses' weren't measured since the begining either, but their data is zero until the first value in 'New_persons_three_doses', since third doses had not been given yet.

To fill the missings from the 'percentage' columns case we will obtain the population of each province and use it after solving problem B (because we need data that will be generated there).

However, to fill the 'New_persons_three_doses', we will just input zeroes in the missing values.

In [27]:
population_province = {}

for current_province in epidemiology_df['province'].unique():
    
    total_number_persons_one_dose     = vacunation_df[vacunation_df['province']==current_province]['new_persons_one_dose'].sum()
    final_percentage_persons_one_dose = vacunation_df[vacunation_df['province']==current_province]['percentage_persons_one_dose'].max()

    population_province[current_province] = total_number_persons_one_dose / final_percentage_persons_one_dose * 100
    
population_province

{'Burgos': 347828.75779106736,
 'Valladolid': 509082.53635826905,
 'Palencia': 157223.49458812363,
 'León': 447229.52868307946,
 'Ávila': 156404.79154812612,
 'TotalCyL': 2391847.508462069,
 'Segovia': 150285.6904653503,
 'Zamora': 166534.37871694405,
 'Salamanca': 318453.8270868207,
 'Soria': 87513.51814894995}

In [28]:
vacunation_df['new_persons_three_doses'] = vacunation_df['new_persons_three_doses'].fillna(0)

### Solving problem B
##### Obtain the accumulated values of each variable from the daily new values
In this step we will also fill the missings from the 'percentage' columns mentioned in problem A, solving the remaining problem.

In [29]:
# Create the new variables for daily data
vacunation_df['total_doses_administered'] = 0

vacunation_df['total_number_persons_one_dose'] = 0
vacunation_df['total_number_persons_two_doses'] = 0
vacunation_df['total_number_persons_three_doses'] = 0

# Obtain the accumulated data by adding the values from consecutive days
for current_province in vacunation_df['province'].unique():
    
    rows_current_province = vacunation_df['province'] == current_province
    data_current_province = vacunation_df[rows_current_province]
    
    vacunation_df.loc[rows_current_province,'total_doses_administered'] = data_current_province['new_doses_administered'].cumsum()
    
    vacunation_df.loc[rows_current_province,'total_number_persons_one_dose'] = data_current_province['new_persons_one_dose'].cumsum()
    vacunation_df.loc[rows_current_province,'total_number_persons_two_doses'] = data_current_province['new_persons_two_doses'].cumsum()
    vacunation_df.loc[rows_current_province,'total_number_persons_three_doses'] = data_current_province['new_persons_three_doses'].cumsum()
    
    vacunation_df.loc[rows_current_province,'percentage_persons_one_dose'] = vacunation_df['total_number_persons_one_dose'] / population_province[current_province] * 100
    vacunation_df.loc[rows_current_province,'percentage_persons_two_doses'] = vacunation_df['total_number_persons_two_doses'] / population_province[current_province] * 100
    vacunation_df.loc[rows_current_province,'percentage_persons_three_doses'] = vacunation_df['total_number_persons_three_doses'] / population_province[current_province] * 100

### Final state of the dataset

In [33]:
vacunation_df

,date,province,new_doses_administered,new_persons_two_doses,new_persons_one_dose,percentage_persons_one_dose,percentage_persons_two_doses,new_persons_three_doses,percentage_persons_three_doses,total_doses_administered,total_number_persons_one_dose,total_number_persons_two_doses,total_number_persons_three_doses
0,2020-11-07,Segovia,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
1,2020-11-07,Soria,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
2,2020-11-07,Zamora,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
3,2020-11-07,TotalCyL,1,0,1,0.000042,0.000000,0.0,0.000000,1,1,0,0
4,2020-11-07,Burgos,0,0,0,0.000000,0.000000,0.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4885,2022-04-12,León,115,34,13,90.491341,88.062835,74.0,64.150281,1047602,404704,393843,286899
4886,2022-04-12,Segovia,84,8,3,87.986421,85.831858,72.0,57.513127,333664,132231,128993,86434
4887,2022-04-12,Soria,46,3,0,91.481867,89.742707,44.0,63.271368,205408,80059,78537,55371
4888,2022-04-12,Burgos,267,140,23,90.564105,88.160623,119.0,58.877248,790092,315008,306648,204792


## Merging of both datasets

In [32]:
df = pd.merge(vacunation_df, epidemiology_df, how='outer', on=['date', 'province']).sort_values(by=['date', 'province']).reset_index(drop=True)
df

,date,province,new_doses_administered,new_persons_two_doses,new_persons_one_dose,percentage_persons_one_dose,percentage_persons_two_doses,new_persons_three_doses,percentage_persons_three_doses,total_doses_administered,total_number_persons_one_dose,total_number_persons_two_doses,total_number_persons_three_doses,total_positives,new_positives,total_discharges,total_deceases,new_discharges,new_deceases
0,2020-03-13,Burgos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,29.0,0.0,1.0,0.0,1.0
1,2020-03-13,León,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,22.0,0.0,0.0,0.0,0.0
2,2020-03-13,Palencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0
3,2020-03-13,Salamanca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,5.0,1.0,1.0,1.0,1.0
4,2020-03-13,Segovia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,6.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,2022-04-19,Soria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28118.0,73.0,1665.0,330.0,2.0,0.0
7656,2022-04-19,TotalCyL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,709807.0,1387.0,41195.0,7369.0,64.0,7.0
7657,2022-04-19,Valladolid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166979.0,213.0,10009.0,1510.0,9.0,1.0
7658,2022-04-19,Zamora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44438.0,120.0,2753.0,565.0,6.0,2.0


Once we merge both datasets we can see a new issue that emerges, which is the temporal difference between the datasets: one began saving data on March 2020, and the other one in November 2020 (not vaccine doses were shot until that moment). Keeping this in mind, we can solve the missing values by inputing zeros, as they represent the reality of the situation.

Nonetheless there is one exception: 'epidemiology_df' has data until the 19th May 2022, while 'vacunation_df' only reaches the 12th May 2022. Thus, we will eliminate this last week.

,date,province,new_doses_administered,new_persons_two_doses,new_persons_one_dose,percentage_persons_one_dose,percentage_persons_two_doses,new_persons_three_doses,percentage_persons_three_doses,total_doses_administered,total_number_persons_one_dose,total_number_persons_two_doses,total_number_persons_three_doses,total_positives,new_positives,total_discharges,total_deceases,new_discharges,new_deceases
0,2020-03-13,Burgos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,29.0,0.0,1.0,0.0,1.0
1,2020-03-13,León,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,22.0,0.0,0.0,0.0,0.0
2,2020-03-13,Palencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0
3,2020-03-13,Salamanca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,5.0,1.0,1.0,1.0,1.0
4,2020-03-13,Segovia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,6.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7586,2022-04-12,TotalCyL,1142.0,395.0,93.0,91.582343,89.014454,668.0,62.330186,5577958.0,2190510.0,2129090.0,1490843.0,700424.0,1705.0,40915.0,7328.0,55.0,6.0
7587,2022-04-12,Valladolid,310.0,168.0,43.0,92.076032,89.564455,101.0,61.868946,1182027.0,468743.0,455957.0,314964.0,165268.0,256.0,9951.0,1502.0,12.0,2.0
7588,2022-04-12,Zamora,106.0,4.0,2.0,91.752827,89.737627,99.0,68.746766,403567.0,152800.0,149444.0,114487.0,43476.0,159.0,2733.0,560.0,0.0,0.0
7589,2022-04-12,Ávila,33.0,19.0,3.0,91.788109,89.611705,3.0,62.634270,368941.0,143561.0,140157.0,97963.0,43343.0,76.0,2442.0,464.0,1.0,0.0


In [41]:
df = df[0:7590].fillna(0)
df

,date,province,new_doses_administered,new_persons_two_doses,new_persons_one_dose,percentage_persons_one_dose,percentage_persons_two_doses,new_persons_three_doses,percentage_persons_three_doses,total_doses_administered,total_number_persons_one_dose,total_number_persons_two_doses,total_number_persons_three_doses,total_positives,new_positives,total_discharges,total_deceases,new_discharges,new_deceases
0,2020-03-13,Burgos,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,90.0,29.0,0.0,1.0,0.0,1.0
1,2020-03-13,León,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,32.0,22.0,0.0,0.0,0.0,0.0
2,2020-03-13,Palencia,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,2020-03-13,Salamanca,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,11.0,5.0,1.0,1.0,1.0,1.0
4,2020-03-13,Segovia,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,6.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7585,2022-04-12,Soria,46.0,3.0,0.0,91.481867,89.742707,44.0,63.271368,205408.0,80059.0,78537.0,55371.0,27635.0,88.0,1657.0,329.0,4.0,0.0
7586,2022-04-12,TotalCyL,1142.0,395.0,93.0,91.582343,89.014454,668.0,62.330186,5577958.0,2190510.0,2129090.0,1490843.0,700424.0,1705.0,40915.0,7328.0,55.0,6.0
7587,2022-04-12,Valladolid,310.0,168.0,43.0,92.076032,89.564455,101.0,61.868946,1182027.0,468743.0,455957.0,314964.0,165268.0,256.0,9951.0,1502.0,12.0,2.0
7588,2022-04-12,Zamora,106.0,4.0,2.0,91.752827,89.737627,99.0,68.746766,403567.0,152800.0,149444.0,114487.0,43476.0,159.0,2733.0,560.0,0.0,0.0
